# Fetch testing

Testing ground for `fetch.py` script

In [1]:
import os

In [9]:
bucket_url = "gs://service-calls-data-lake"
fname = "calls.csv"
csvpath = os.path.join(bucket_url, fname)
csvpath

'gs://service-calls-data-lake/calls.csv'

Test whether gcs object exists

In [8]:
from pathlib import Path

In [7]:
from google.cloud.storage import Client, Blob

In [4]:
gcs = Client(project="de-zoom-83")

In [6]:
for b in gcs.list_buckets():
    print(b)

<Bucket: dataproc-staging-us-west1-861494848006-wheyfjcm>
<Bucket: dataproc-temp-us-west1-861494848006-o2e0uj5a>
<Bucket: de-zoom-prefect>
<Bucket: dtc_data_lake_de-zoom-83>
<Bucket: service-call-tf-states>
<Bucket: service-calls-data-lake>
<Bucket: tf-states-de-zoom>


In [14]:
bucket_name = "service-calls-data-lake"
fpath = Path("../data/city-wards-boundary.geojson")
# use os.path.join to avoid messing with gs:// double slash
# do not need bucket name in beginning
gcsobj = os.path.join("geojson", fpath.name)
bucket_client = gcs.bucket(bucket_name)
blob_client = bucket_client.blob(gcsobj)
blob_client.upload_from_filename(fpath)

In [15]:
from tempfile import TemporaryDirectory

In [16]:
with TemporaryDirectory() as tmp1:
    print(tmp1)
    with TemporaryDirectory() as tmp2:
        print(tmp2)

/tmp/tmpdlxb1lmf
/tmp/tmp1juacnbk


In [17]:
import pandas as pd

In [18]:
df = pd.DataFrame([[4, 9]] * 3, columns=["A", "B"])
gcspq = f"gs://{bucket_name}/raw/pq/test.parquet"
df.to_parquet(path=gcspq)